In [ ]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

#판다스로 어휘 문법 등급 목록 가져오기
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/텍마과제/2017년 국제 통용 한국어 표준 교육과정 적용 연구(4단계) 어휘, 문법 등급 목록.xlsx", sheet_name = [0,1])

#어휘와 문법의 구성 요소만 확인할 수 있는 데이터프레임 생성
df_voca = pd.concat([df[0]['어휘'], df[1]['대표형']])

#확인
df_voca

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


0          가게
1        가격02
2        가구02
3         가깝다
4        가다01
        ...  
331      이라고는
332    -는다던가1
333     -으래서야
334     -으리라고
335      -자면2
Length: 10971, dtype: object

In [ ]:
#값 내 숫자 제거
df_voca = df_voca.replace(r'\d+', '', regex=True)

#숫자 제거 확인
df_voca

0         가게
1         가격
2         가구
3        가깝다
4         가다
       ...  
331     이라고는
332    -는다던가
333    -으래서야
334    -으리라고
335      -자면
Length: 10971, dtype: object

In [ ]:
#코모란 설치
!pip install PyKomoran

In [ ]:
#코모란 라이브러리 가져오기 및 확인
from PyKomoran import *
komoran = Komoran("EXP")
print(komoran.get_plain_text("KOMORAN은 한국어 형태소 분석기입니다."))

KOMORAN/SL 은/JX 한국어/NNP 형태소/NNP 분석기/NNG 이/VCP ㅂ니다/EF ./SF


In [ ]:
#한국어 불용어 사전 가져오기
stop = pd.read_table("/content/drive/MyDrive/텍마과제/koreanStopwords.txt", names =["단어", "형태소", "수치"])
print(stop)

    단어  형태소        수치
0    이  VCP  0.018280
1    있   VA  0.011699
2    하   VV  0.009774
3    것  NNB  0.009733
4    들  XSN  0.006898
..  ..  ...       ...
95   원  NNB  0.000492
96   잘  MAG  0.000491
97  통하   VV  0.000487
98  소리  NNG  0.000486
99   놓   VX  0.000483

[100 rows x 3 columns]


In [ ]:
#불용어 사전에서 필요한 정보만 추출하기
stop['단어']

0      이
1      있
2      하
3      것
4      들
      ..
95     원
96     잘
97    통하
98    소리
99     놓
Name: 단어, Length: 100, dtype: object

In [ ]:
#어휘 문법 사전의 구성 요소들을 형태소 형태로 변경 + 불용어 제거
stopwords = [[stop['단어'], '-']]

voca_list = []

for w in df_voca:
  newdf = komoran.get_morphes_by_tags(w, tag_list=['NP', 'NNG', 'XSA', 'EP', 'EF', 'JX', 'NNP', 'JKB', 'XSV', 'EC', 'VX', 'ETM', 'XSN', 'VCP', 'VV', 'VA', 'JKS', 'MAG', 'XR', 'JC', 'NA', 'VCN', 'MM', 'ETN', 'MAJ', 'JKG', 'JKC', 'IC', 'NNB', 'NR', 'JKO', 'XPN', 'JKQ'])

  for w in newdf:
    if w not in stopwords:
      voca_list.append(w)

#형태소 형태로 전환된 새로운 어휘 문법 데이터프레임 생성
vocadf = pd.Series(voca_list)

#확인
print('상위 20개:\n',vocadf.head(20))
print('상위 20개:\n',vocadf.tail(20))

#데이터 프레임 저장
vocadf.to_csv('/content/drive/MyDrive/텍마과제/vocadf.csv', index=False)

상위 20개:
 0       가
1       게
2      가격
3      가구
4      가깝
5       다
6       가
7       다
8     가르치
9       다
10     가방
11     가볍
12      다
13     가수
14     가요
15    가운데
16      가
17      을
18     가장
19     가족
dtype: object
상위 20개:
 15260    으리라는
15261       를
15262      막론
15263       하
15264       고
15265       이
15266       어
15267      치우
15268       다
15269       이
15270     는다는
15271     이라고
15272       는
15273       이
15274    는다던가
15275      으래
15276       서
15277      어야
15278    으리라고
15279      자면
dtype: object
